# Sistemas de Recomendacion avanzados

Diego Galeano, Ph.D.

Material basado en: https://developers.google.com/machine-learning/recommendation/labs/movie-rec-programming-exercise

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
import numpy as np
import copy
import random
import scipy.io
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

# If you want to have direct access to the datasets and codes you can clone the following github repository    
! git clone https://github.com/saminehbagheri/Recommender-System.git
%cd Recommender-System  

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Cloning into 'Recommender-System'...
remote: Enumerating objects: 66, done.
remote: Total 66 (delta 0), reused 0 (delta 0), pack-reused 66
Unpacking objects: 100% (66/66), done.
/content/Recommender-System


### Leemos el Movielens dataset

In [ ]:
mat = scipy.io.loadmat('ex8_movies.mat')
movie_names = pd.read_csv('movie_ids.txt',delimiter=';',header=None)[1]
Y=mat['Y']
R=mat['R']
num_user=Y.shape[1]
num_movie=Y.shape[0]
density= 100*np.sum(np.where(Y > 0,1,0))/(num_user*num_movie)
print("numero de usuarios:"+str(num_user))
print("numero de peliculas:"+str(num_movie))
print("densidad del rating matrix: "+str(density) + '%')

numero de usuarios:943
numero de peliculas:1682
densidad del rating matrix: 6.304669364224532%


## Matrix factorizacion: collaborative filtering

Recordando el sistema de recomendación basado en contenido, la idea era describir cada item y cada usuario con un vector de características importantes. En nuestro ejemplo de la clase anterior, el vector de características que usamos tenía solo tres elementos, pero somos conscientes de que hay criterios mucho más importantes que esos tres para capturar nuestro interés por una película y, a veces, estas características pueden ser más complicadas que simplemente el género de la película.

 Matrix Factorization supone que podemos describir una película y un usuario en forma de un vector de características. La idea principal de la factorización matricial es encontrar el vector de característica de usuario adecuado $ \vec {x}_i $ y el vector de característica de elemento $ \vec{\theta}_j $ para todos los usuarios $ i = 1 \cdots n_u $ y películas $ j = 1 \cdots n_m $ que sus productos punto dan una buena estimación de la calificación que el $ i $ -ésimo usuario daría a la $ j $ -ésima película $ y_ {ij} $.

 **Porque este metodo se llama decomposicion de matrices?** Porque el rating que un usuario da a una pelicula se modela como el producto escalar entre el vector de caracteristicas del usuario y el vector de caracteristicas de la pelicula, es decir: $\vec{x}_i\cdot\vec{\theta}_j=y_{ij}$. Supongamos que apilamos todos los vectores de características del usuario en la matriz de características del usuario $\mathbf{X}=\begin{bmatrix}-\vec{x}_1^T-\\ -\vec{x}_2^T- \\ \vdots \\-\vec{x}_{n_u}^T- \end{bmatrix}_{n_u \times n_f}$  y todos los vectores de características de la película juntos en la matriz de características de la película $\mathbf{\Theta}=\begin{bmatrix}-\vec{\theta}_1^T-\\ -\vec{\theta}_2^T- \\ \vdots \\-\vec{\theta}_{n_m}^T- \end{bmatrix}_{n_m \times n_f}$. Entonces, la matriz de calificación se puede determinar de la siguiente manera:

 
\begin{equation}\mathbf{\Theta} \cdot \mathbf{X}^T=\mathbf{R}\end{equation}
\begin{equation}\begin{bmatrix}- \vec{\theta}_1^T-\\ - \vec{\theta}_2^T- \\ \vdots \\- \vec{\theta}_{n_m}^T- \end{bmatrix} \cdot \begin{bmatrix}|&|&\cdots&|\\
               \vec{x_1}&\vec{x_2}&\cdots&\vec{x}_{n_u}\\
               |&|&\cdots&|\end{bmatrix}=\mathbf{R}\end{equation}

Sería perfecto si tuviéramos las matrices de características adecuadas, pero no las tenemos. Lo que en realidad tenemos es la matriz de calificación $ \mathbf{Y}$ e intentamos *aprender* las matrices de características de $ \mathbf{Y}$. El nombre **factorización de matrices** proviene de este punto que este algoritmo tiende a encontrar matrices de características razonablemente óptimas al factorizar la matriz de calificación $ \mathbf{Y}$.

**¿Cómo funciona la factorización de matrices?** Supongamos que iniciamos las matrices de características con valores completamente aleatorios. El producto escalar del vector de características de usuario $ i$ -ésimo y el vector de características de película $ j $ -ésimo dará $ p_{ij} $ que probablemente sea muy diferente a la calificación real dada $ y_{ij} $. La idea es encontrar las matrices de características de manera que el error $ | p_{ij} -y_{ij} | $ sea lo más mínimo posible para todas las calificaciones dadas. En otras palabras, la factorización matricial se convirtió en un problema de optimización de minimizar una función de costo que es la suma de todos los errores $ \color {green}{\text{al cuadrado}} $ para todas las celdas con una calificación. La función de costo a minimizar se define de la siguiente manera:


\begin{equation}J(\mathbf{\Theta},\mathbf{X})=\frac{1}{2} \sum\limits_{(i,j):r(i,j)=\{1\}} (\vec{\theta}_{j} \vec{x}_{i}-y_{i,j})^2\end{equation}

**¿Cómo resolver un problema de optimización de este tipo?** Podemos pasar la función de costo con un punto de partida aleatorio a un optimizador y esperar hasta que el optimizador encuentre una solución óptima, pero esto llevará un tiempo insoportablemente largo ya que el problema de optimización es dimensional muy alto. ¿Puede adivinar cuál es la dimensión de entrada de este problema de optimización? $ (n_u + n_m) * n_f $, donde $ n_f $ es el número de caracteristicas.

Es más lógico calcular los gradientes hacia la solución óptima de manera iterativa mediante un descenso de gradiente o métodos de gradiente conjugado. Es muy sencillo calcular los gradientes ya que nuestra función de costo es una función cuadrática.


## The Gradients
 \begin{equation}\frac{\partial J}{\partial x_i^{(k)}}=\sum\limits_{j:r(i,j)=1}(\vec{\theta}_{j} \vec{x}_{i}-y_{i,j})\theta^{(k)}_j,\end{equation}
 \begin{equation}\frac{\partial J}{\partial \theta_j^{(k)}}=\sum\limits_{i:r(i,j)=1}(\vec{\theta}_{j} \vec{x}_{i}-y_{i,j})x^{(k)}_i,\end{equation}

 
donde $x^{(k)}_j$ es el elemento $k$-esimo del $i$-esimo vector de caracteristicas del usuario $\vec{x}_i$. 

Los pasos principales del algoritmo de factorización matricial se pueden resumir a continuación:

1. Inicialice $ \mathbf {\Theta} $ y $ \mathbf{X} $ con números pequeños aleatorios
2. Minimice la función de costo $ J (\mathbf{\Theta}, \mathbf{X}) $
3. Utilice las matrices de características optimizadas para predecir

<a id='MFPy'> </a>
# Implementación de factorización matricial

## Función de inicialización de parámetros
Esta función simplemente obtiene el número de usuarios, el número de elementos y el número de características como entrada y devuelve la matriz de características del usuario iniciada aleatoriamente y la matriz de características del elemento.

In [ ]:
def initilizeFeat(nu,ni,nf,seed=42):
    '''
    Inicializacion de las matrices de caracteristicas.
    '''
    # semilla de randomizacion
    np.random.seed(seed)
    # inicializacion aleatoria de matrices de caracteristicas
    Theta = np.random.rand(nu,nf)*0.05
    X = np.random.rand(ni,nf)*0.05
    return X, Theta

## Funciones auxiliares
Tenemos dos funciones auxiliares. Una para aplanar las matrices de características en una matriz 1-D, y otra hace lo inverso.

In [ ]:
def flatterRev(x,nu,ni,nf):
    '''
    Convierte un vector 1-D a las matrices de caracteristicas X y Theta.
      x: vector 1-D.
      nu: numero de usuarios.
      ni: numero de items.
      nf: numero de caracteristicas.    
    ''' 
    X=x[0:ni*nf].reshape((ni,nf),order='F')
    Theta=x[ni*nf:].reshape((nu,nf),order='F')
    return X,Theta

def flatter(X, Theta):
    '''
    Convierte las matrices de caracteristicas a un vector 1-D.
      X: matriz de caracteristicas de usuarios.
      Theta: matriz de caracteristicas de peliculas.
    ''' 
    x=np.concatenate([X.reshape(X.shape[0]*X.shape[1],order='F'),Theta.reshape(Theta.shape[0]*Theta.shape[1],order='F')])
    return(x)
    

# Función de costo
La función de costo es simplemente la implementación de la fórmula de la función de costo que hemos discutido anteriormente en una forma vectorizada para evitar bucles for anidados para las sumas. La única diferencia es un término adicional vinculado con el parámetro $ \lambda $, el parámetro de regularización.

In [ ]:
def costFunc(X,Theta,R,M,la=0):
    '''
    Funcion auxiliar de la funcion de costo.

    '''
    R=np.ma.array(R, mask=M)
    e=0.5*np.sum(np.power((np.dot(Theta,X.T)-R),2))+la*0.5*np.sum(np.power(Theta, 2))+la*0.5*np.sum(np.power(X, 2))
    return(e/np.sum(M==False))


def CF(x,R,M,nu,ni,nf,la=0):
    '''
    Funcion de costo con termino de regularizacion.
      x: 1-D vector que contiene X y Theta.
      R: matriz de ratings.
      M: matriz de enmascaramiento para solo optimizar en los ratings observados.
      nu: numero de usuarios.
      ni: numero de items.
      nf: numero de caracteristicas. 
      la: lambda, termino de regularizacion L2.   
    ''' 
    X, Theta=flatterRev(x,nu,ni,nf)
    error=costFunc(X,Theta,R,M,la=la)
    return error

# Gradiente

In [ ]:
def gradFunc(x,R,M,nu,ni,nf,la=0 ):
    '''
    Retorna los gradientes para el optimizador.

    '''
    X, Theta=flatterRev(x,nu,ni,nf)
    R=np.ma.array(R, mask=M)
    e=np.dot(Theta,X.T)-R
    TG=np.dot(e,X)+la*Theta
    XG=np.dot(e.T,Theta)+la*X
    grads=np.concatenate([XG.reshape(XG.shape[0]*XG.shape[1],order='F'),TG.reshape(TG.shape[0]*TG.shape[1],order='F')])
    return grads/np.sum(M==False)

## Función de entrenamiento

La función de entrenamiento en realidad es solo aplicar un algoritmo de optimización en la función de costo. Usamos un optimizador de gradiente conjugado incorporado de la biblioteca scipy. Es posible que desee probar diferentes métodos.

In [ ]:
def trainMF(R,M,nf,la=0,seed=42):
    nu=R.shape[0]
    ni=R.shape[1]
    R=np.ma.array(R, mask=M)
    X, Theta=initilizeFeat(nu,ni,nf,seed=seed)
    x=flatter(X, Theta)  
    
    res = minimize(CF, x, args=(R,M,nu,ni,nf,la), method='CG',jac=gradFunc,options={ 'disp': True,'gtol':1e-5})
    MSE=CF(res.x,R,M,nu,ni,nf,la)
    return(MSE, res,nu,ni,nf)

# Funcion de prediccion

In [ ]:
def Predict(res,nu,ni,nf,la=0):
    X, Theta=flatterRev(res.x,nu,ni,nf)
    predict=np.dot(Theta,X.T)
    return(predict)

# Construyendo el modelo del sistema de recomendación

In [ ]:
runEXAMPLE=True
def buildRSModel(R,M,mu=None, nf=10,la=0,seed=42, movie_names=None):
    trainR=copy.copy(R)
    trainM=copy.copy(M)

    trainR=np.ma.array(trainR, mask=trainM)
    if mu is None:
        mu=np.average(trainR,axis=1)
    trainR=trainR-mu[:,None]
    trainingError, res,nu,ni,nf=trainMF(trainR,M,nf=nf,la=la,seed=seed)
    model={'trainingError': trainingError, 'res':res,'nu':nu,'ni':ni,'nf':nf, 'la':la, 'movie_names':movie_names, 'mu':mu,
          'R':R,'M':M}
    return model

#Ejemplo
if runEXAMPLE:
    R=mat['Y']
    M=mat['R']
    trainR=copy.copy(R)
    trainM= (M==0)
    mymodel=buildRSModel(R=trainR,M=trainM,mu=None, nf=100,la=0,seed=42, movie_names=movie_names)
    #print(mymodel)

         Current function value: 0.158532
         Iterations: 16
         Function evaluations: 335
         Gradient evaluations: 324


# Predicción para el usuario X

In [ ]:
runEXAMPLE=True
def predictForUserX(user_Id,model,movie_Id=None):
    trainingError=model['trainingError']
    res=model['res']
    nu=model['nu']
    ni=model['ni']
    nf=model['nf']
    la=model['la']
    movie_names=model['movie_names']
    mu=model['mu']
    R=model['R']
    M=model['M']
    mypredict=Predict(res,nu,ni,nf,la=0)
    mydata=pd.DataFrame()
    Pred=mypredict[:,user_Id]+mu[user_Id]
    mydata['names']=movie_names
    mydata['predictedRating']=Pred
    mydata['originalrating']=R[:,user_Id]
    mydata=mydata.sort_values(by=['predictedRating'], ascending=False)
    output=mydata[mydata['originalrating'] == 0]
    return(output)

#Example
if runEXAMPLE:
    user_id = 125
    print(predictForUserX(user_id,mymodel,movie_Id=None).head())

                              names  predictedRating  originalrating
471               Dragonheart(1996)         4.268084               0
679        Kull the Conqueror(1997)         4.233533               0
320                    Mother(1996)         4.195186               0
627                  Sleepers(1996)         4.182644               0
221  Star Trek: First Contact(1996)         4.162360               0


# Ingrese su propia calificación

Puede usar esta función para ingresar su propia calificación y ver lo que sugiere el sistema. Si no configura el parámetro model, se utilizarán los valores predeterminados de nf = 100, la = 0.1.

In [ ]:
def weRecommend(myratings,modelparam=None):
    movie_names = pd.read_csv('movie_ids.txt',delimiter=';',header=None)[1]
    mat = scipy.io.loadmat('ex8_movies.mat')
    print("Reading the data")
    R=mat['Y']
    M=mat['R']
    trainR=copy.copy(R)
    trainM= (M==0)
    num_user=R.shape[1]
    num_movie=R.shape[0]
    
    
    myratings=myratings.sort_values(by=['names'], ascending=False)
    movies=copy.copy(movie_names)
    movies=movies.sort_values( ascending=False)
    indices=movies[movies.isin( myratings['names'])].index
    newuserratingR=np.zeros(num_movie)
    newuserratingM=np.zeros(num_movie)
    newuserratingR[indices]=myratings['rating']
    newuserratingM[indices]=1
    
    newuserratingM= (newuserratingM==0)
    trainR=np.concatenate((newuserratingR[:,None],trainR),axis=1)
    trainM=np.concatenate((newuserratingM[:,None],trainM),axis=1)
    
    print("Training the Recommender System...")
    if modelparam is None:
        mymodel=buildRSModel(R=trainR,M=trainM,mu=None, nf=100,la=0.1, movie_names=movie_names)
    else:
        nf=modelparam['nf']
        la=modelparam['la'] 
        mymodel=buildRSModel(R=trainR,M=trainM,mu=None, nf=nf,la=la, movie_names=movie_names)
    print("Training is successfully finished")   
    bests=predictForUserX(0,mymodel,movie_Id=None).head(15)
    worsts=predictForUserX(0,mymodel,movie_Id=None).tail(15)
    print("Predicting you're ratings:")
    bests=bests.iloc[:, :-1]
    worsts=worsts.iloc[:, :-1]
    output={'bests':bests,'worsts':worsts}
    return output


In [ ]:
 movie_names = pd.read_csv('movie_ids.txt',delimiter=';',header=None)[1]

In [ ]:
df = pd.DataFrame()
df['rating'] = [5,4,5,5,5]
df['names'] = ['Toy Story(1995)', 'Batman Forever(1995)', 'Ace Ventura: Pet Detective(1994)', 'Lion King  The(1994)', 'Mask  The(1994)' ]

In [ ]:
 weRecommend(df)

Reading the data
Training the Recommender System...
         Current function value: 0.164909
         Iterations: 16
         Function evaluations: 274
         Gradient evaluations: 263
Training is successfully finished
Predicting you're ratings:


{'bests':                                    names  predictedRating
 94                         Aladdin(1992)         4.137037
 587           Beauty and the Beast(1991)         4.103507
 741                         Ransom(1996)         4.080346
 236                  Jerry Maguire(1996)         4.039222
 185            Blues Brothers  The(1980)         4.032696
 392                 Mrs. Doubtfire(1993)         4.028297
 293                      Liar Liar(1997)         4.027884
 158                 Basic Instinct(1992)         4.027414
 767                         Casper(1995)         4.020870
 229  Star Trek IV: The Voyage Home(1986)         4.020013
 635           Escape from New York(1981)         4.018811
 595   Hunchback of Notre Dame  The(1996)         4.017570
 6                   Twelve Monkeys(1995)         4.015794
 68                    Forrest Gump(1994)         4.013471
 95      Terminator 2: Judgment Day(1991)         4.012567,
 'worsts':                              names 

# Optimizar los parametros del sistema de recomendacion

## Función de error de prueba

Existen diferentes enfoques para evaluar el desempeño de un sistema de recomendación en los datos de prueba. Un método consiste en calcular el error cuadrático medio de los datos de prueba. Otro enfoque es medir algún tipo de precisión de predicción. En la siguiente celda, definimos la precisión como el porcentaje de las calificaciones pronosticadas que tienen un error de 1 o menos.

In [ ]:
def testMF(tR,tM,predict):
    tR=np.ma.array(tR, mask=tM)
    e=np.abs(tR-predict)
    testMSE=np.sum(np.power(e,2))/np.sum(tM==False)
    return(testMSE)

In [ ]:
def splitMatrix(R,M,testPer):
    trainPer=1-testPer
    num_user=R.shape[1]
    num_movie=R.shape[0]
    overallRating=np.sum(M)
    testsize=testPer*overallRating
    testsize=testsize.astype(int)


    #split tarining and test dataset
    random.seed( 9273482 )
    ind1, ind2=np.where(M==1)
    testSamples=random.sample(range(ind1.shape[0]), testsize)
    testInd1=ind1[testSamples]
    testInd2=ind2[testSamples]

    trainR=copy.copy(R)
    trainM=copy.copy(M)
    trainR[testInd1,testInd2]=0
    trainM[testInd1,testInd2]=0


    M= (trainM==0)
    trainR=np.ma.array(trainR, mask=M)
    mu=np.average(trainR,axis=1)

    testR=copy.copy(R)
    testM=np.zeros(shape = (testR.shape[0],testR.shape[1]))
    testM[testInd1,testInd2]=1
    tM=(testM==0)
    testR=testR*testM
    testR=np.ma.array(testR, mask=tM)
    return trainR, M, testR, tM, mu

In [ ]:
DORUN=True

trainR, M, testR, tM, mu=splitMatrix(Y,R,0.1)
NF=[1,5,10,20]
myseed=5623
if DORUN:
    trainR=trainR-mu[:,None]
    testR=testR-mu[:,None]

    for nf in NF:        
        trainingError, res,nu,ni,nf=trainMF(trainR,M,nf=nf,la=0.1,seed=myseed)
        mypredict=Predict(res,nu,ni,nf,la=0)
        zeropredict=np.zeros(shape = (mypredict.shape[0],mypredict.shape[1]))        
        testError=testMF(testR,tM,mypredict)
        
        print('[nf='+str(nf)+']'+'Training Error:'+str(trainingError))
        print('[nf='+str(nf)+']'+'Test Error:'+str(testError))
     

         Current function value: 0.467095
         Iterations: 4
         Function evaluations: 74
         Gradient evaluations: 63
[nf=1]Training Error:0.46709498777841846
[nf=1]Test Error:0.995016231120181
         Current function value: 0.466449
         Iterations: 3
         Function evaluations: 103
         Gradient evaluations: 91
[nf=5]Training Error:0.46644884839687906
[nf=5]Test Error:0.9962897332608236
         Current function value: 0.463523
         Iterations: 2
         Function evaluations: 59
         Gradient evaluations: 48
[nf=10]Training Error:0.4635231472060253
[nf=10]Test Error:0.9924336179545064
         Current function value: 0.466339
         Iterations: 3
         Function evaluations: 92
         Gradient evaluations: 80
[nf=20]Training Error:0.4663389769881152
[nf=20]Test Error:0.9980363109345022
